## Data Wrangling

Data exploration, dealing with missing values, reshaping data (one hot encoding), pivot tables, joins, grouping and aggregating
filtering data, making descriptive columns, element-wise conditional operations.

In [17]:
# Data types and the data
import pandas as pd

df = pd.read_csv('auto_mpg.csv')
print(df.dtypes)
df.head(3)

mpg             float64
cylinders         int64
displacement    float64
horsepower      float64
weight            int64
acceleration    float64
model_year        int64
origin           object
name             object
dtype: object


,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick
2,18.0,8,318.0,150.0,3436,11.0,70,usa,chrysler


In [18]:
# Statistical data for all numerical columns

df.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year
count,398.000000,398.000000,398.000000,392.000000,398.000000,398.000000,398.000000
mean,23.514573,5.454774,193.425879,104.469388,2970.424623,15.568090,76.010050
std,7.815984,1.701004,104.269838,38.491160,846.841774,2.757689,3.697627
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000
25%,17.500000,4.000000,104.250000,75.000000,2223.750000,13.825000,73.000000
50%,23.000000,4.000000,148.500000,93.500000,2803.500000,15.500000,76.000000
75%,29.000000,8.000000,262.000000,126.000000,3608.000000,17.175000,79.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000


In [19]:
# Statistical data for categorical columns

df.groupby(by = ['origin', 'name']).size()

origin  name         
europe  audi              7
        bmw               3
        fiat              9
        mercedes-benz     3
        opel              9
        peugeot           8
        renault           5
        saab              4
        volkswagen       15
        volvo             7
japan   datsun           24
        honda            14
        mazda            13
        nissan            1
        toyota           27
usa     buick            73
        chevrolet        58
        chrysler         45
        ford             73
dtype: int64

In [20]:
# Dealing with missing values

total_missing_values = df.isna().sum()
print("Total Missing Values: \n", total_missing_values)

Total Missing Values: 
 mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model_year      0
origin          0
name            0
dtype: int64


#### Possible options for filling the missing values:  zero, mean/max/median by column, some string or drop them by row. 

#### Since there are few missing values for 'horsepower' attribute, we can drop the rows.

In [21]:
# Dropping missing values

df = df.dropna(axis = 0, how = 'any')

updated_missing_values = df.isna().sum()
print("Updated No. of Missing Values: \n", updated_missing_values)

Updated No. of Missing Values: 
 mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model_year      0
origin          0
name            0
dtype: int64


In [22]:
# Display unique values in each column

df.nunique()

mpg             127
cylinders         5
displacement     81
horsepower       93
weight          346
acceleration     95
model_year       13
origin            3
name             19
dtype: int64

In [23]:
# Shape of the dataframe (No. of rows, No. of columns)

df.shape

(392, 9)

#### There are total 392 rows but there are 346 unique values for 'weight' attribute, for example. 

In [24]:
# Viewing the repeated values

df[df.duplicated(subset = 'weight', keep = False)].sort_values('weight')

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
198,33.0,4,91.0,53.0,1795,17.4,76,japan,honda
181,33.0,4,91.0,53.0,1795,17.5,75,japan,honda
245,36.1,4,98.0,66.0,1800,14.4,78,usa,ford
248,36.1,4,91.0,60.0,1800,16.4,78,japan,honda
218,36.0,4,79.0,58.0,1825,18.6,77,europe,renault
...,...,...,...,...,...,...,...,...,...
292,18.5,8,360.0,150.0,3940,13.0,79,usa,chrysler
190,14.5,8,351.0,152.0,4215,12.8,76,usa,ford
187,17.5,8,305.0,140.0,4215,13.0,76,usa,chevrolet
91,13.0,8,400.0,150.0,4464,12.0,73,usa,chevrolet


In [25]:
# Count of each weight value

repeat_value = df.groupby(by = 'weight').size().sort_values(ascending = False)
print(repeat_value)

weight
1985    4
2130    4
2265    3
2945    3
2300    3
       ..
3399    1
3415    1
3420    1
3430    1
1613    1
Length: 346, dtype: int64


#### In the above result, it can be seen that weight value '1985' is repeated 4 times.

In [28]:
# Filtering data

# To remove the data for weight that shows up more than 2 times

filtered_data = repeat_value[repeat_value >= 2].to_frame().reset_index()
filtered_df = df[ ~ df.weight.isin(filtered_data.weight)]

# Count of each weight value after deletion

repeat_value = df.groupby(by = 'weight').size().sort_values(ascending = False)
print(repeat_value)

weight
1985    4
2130    4
2265    3
2945    3
2300    3
       ..
3399    1
3415    1
3420    1
3430    1
1613    1
Length: 346, dtype: int64


In [29]:
# Reshaping Data

# 1. Origin column is categorical - performing one hot encoding

df = pd.concat([df, pd.get_dummies(df['origin'], prefix = 'region')], axis = 1).drop(['origin'], axis=1)
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,name,region_europe,region_japan,region_usa
0,18.0,8,307.0,130.0,3504,12.0,70,chevrolet,0,0,1
1,15.0,8,350.0,165.0,3693,11.5,70,buick,0,0,1
2,18.0,8,318.0,150.0,3436,11.0,70,chrysler,0,0,1
3,16.0,8,304.0,150.0,3433,12.0,70,buick,0,0,1
4,17.0,8,302.0,140.0,3449,10.5,70,ford,0,0,1


In [33]:
# 2. Row-wise operations: creating a new column based on the values for two attributes

def car_type_displacement_weight(row):
    if ((row['displacement'] >= 200) & (row['weight'] >= 2500)):
        return('heavy')
    else:
        return('light')

df['car_type'] = df.apply(lambda x: car_type_displacement_weight(x), axis=1)
df.to_csv("auto_mpg_cleaned.csv", index = False)
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,name,region_europe,region_japan,region_usa,car_type
0,18.0,8,307.0,130.0,3504,12.0,70,chevrolet,0,0,1,heavy
1,15.0,8,350.0,165.0,3693,11.5,70,buick,0,0,1,heavy
2,18.0,8,318.0,150.0,3436,11.0,70,chrysler,0,0,1,heavy
3,16.0,8,304.0,150.0,3433,12.0,70,buick,0,0,1,heavy
4,17.0,8,302.0,140.0,3449,10.5,70,ford,0,0,1,heavy
